In [1]:
from google.colab import files

In [14]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install ultralytics

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.2/645.2 kB 10.0 MB/s eta 0:00:00


In [15]:
from ultralytics import YOLO, checks, hub
import os
import cv2
import csv

In [25]:
file = files.upload()


Saving 1.mp4 to 1 (2).mp4
None


In [28]:
list(file.keys())[0]

'1 (2).mp4'

In [16]:
def recognize(filename):
    #VIDEOS_DIR = os.path.join('.', 'videos')
    #video_path = os.path.join(VIDEOS_DIR, filename)
    video_path_out = '{}_out.mp4'.format(filename)

    cap = cv2.VideoCapture(filename)
    ret, frame = cap.read()
    H, W, _ = frame.shape
    out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'mp4v'), int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

    model_path = 'weapon_yolo8.v1i.pt'
    # Load a model
    model = YOLO(model_path)  # load a custom model

    threshold = 0.2
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("--- fps --- {}".format(fps))
    cases = 0
    cur_frame = 0
    cur_hour = 0
    cur_min = 0
    cur_sec = 0
    cur_frame_buffer = 0
    timestamps = list()
    last_frame_detected = 0

    while ret:
        # work with time
        cur_frame += 1
        cur_frame_buffer += 1
        if (cur_frame_buffer >= fps):
            cur_sec += 1
            cur_frame_buffer = 0
        else:
            out.write(frame)
            ret, frame = cap.read()
            continue

        if (cur_sec > 59):
            cur_min += 1
            cur_sec = 0
        if (cur_min > 59):
            cur_min = 0
            cur_hour += 1

        results = model(frame)[0]

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result

            if (score > threshold):
                # Detected
                last_frame_detected = cur_frame
                cases += 1
                timestamps.append("{}:{}".format(cur_min, cur_sec))

                print("---------------------------------------------------------------------------------")
                print("frame {} ==== time {}:{} ====".format(cur_frame, cur_min, cur_sec))
                print(result)
                print("---------------------------------------------------------------------------------")
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
                cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

        out.write(frame)
        ret, frame = cap.read()

    # decision format
    decision = list()
    decision.append(filename)
    decision.append(cases)
    decision.append(timestamps)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return decision


In [29]:
def button_function():
    submission = "result.csv"
    with open(submission, mode="w", encoding='utf-8') as w_file:
        file_writer = csv.writer(w_file, delimiter=";", lineterminator="\r")
        file_writer.writerow(["filename", "cases_count", "timestamps"])
        # f = os.path.join(directory, "1.mp4")

        decision = recognize(list(file.keys())[0])  # uploaded file
        print(decision)
        file_writer.writerow(decision)

In [30]:
button_function()

--- fps --- 25.0



0: 384x640 (no detections), 2879.9ms
Speed: 17.9ms preprocess, 2879.9ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2633.9ms
Speed: 3.7ms preprocess, 2633.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2947.1ms
Speed: 5.3ms preprocess, 2947.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3232.9ms
Speed: 7.0ms preprocess, 3232.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2633.3ms
Speed: 3.5ms preprocess, 2633.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2649.9ms
Speed: 3.9ms preprocess, 2649.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3431.5ms
Speed: 4.1ms preprocess, 3431.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2799.2ms
S

---------------------------------------------------------------------------------
frame 5700 ==== time 3:48 ====
[1191.1226806640625, 729.6823120117188, 1232.162841796875, 806.8292236328125, 0.653812825679779, 3.0]
---------------------------------------------------------------------------------



0: 384x640 (no detections), 3957.8ms
Speed: 4.2ms preprocess, 3957.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2650.6ms
Speed: 3.6ms preprocess, 2650.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2671.3ms
Speed: 4.3ms preprocess, 2671.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2652.2ms
Speed: 6.2ms preprocess, 2652.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3906.6ms
Speed: 4.2ms preprocess, 3906.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2693.7ms
Speed: 3.8ms preprocess, 2693.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2633.7ms
Speed: 3.0ms preprocess, 2633.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2831.9ms
Spe

---------------------------------------------------------------------------------
frame 6475 ==== time 4:19 ====
[1100.0635986328125, 759.3310546875, 1170.833251953125, 840.2926025390625, 0.7330393195152283, 3.0]
---------------------------------------------------------------------------------



0: 384x640 (no detections), 2729.8ms
Speed: 6.8ms preprocess, 2729.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2636.5ms
Speed: 4.4ms preprocess, 2636.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2639.9ms
Speed: 3.6ms preprocess, 2639.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3935.0ms
Speed: 3.7ms preprocess, 3935.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2653.4ms
Speed: 4.1ms preprocess, 2653.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2636.9ms
Speed: 4.9ms preprocess, 2636.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 2664.2ms
Speed: 3.9ms preprocess, 2664.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3907.4ms
Spe

['1 (2).mp4', 2, ['3:48', '4:19']]
